## In this Notebook, we will create a basic Q&A chain and Agent over a SQL database

### Architecture

At a high-level, the steps of any SQL chain and agent are:

- `Convert question to SQL query using the LLM Model.`
- `Execute the SQL query.`
- `getting the Model to respond to user input using the query results.`

In [6]:
from langchain.utilities import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
llm = ChatOpenAI(temperature=0)

In [9]:
host = '0.0.0.0'
port = '5432'
username = 'odin'
password = 'odin'
db_name = 'analytics'
postgres_connection = f"postgresql://{username}:{password}@{host}:{port}/{db_name}"

db = SQLDatabase.from_uri(postgres_connection, sample_rows_in_table_info=2)
chain = create_sql_query_chain(llm, db)

In [5]:
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT count(*) FROM customer LIMIT 10;")

mysql
['customer', 'employee', 'inventoryitem', 'itemsize', 'manufacturer', 'product', 'purchase', 'purchaseitem', 'sale', 'saleitem', 'your_table_name']


'[(4417,)]'

In [10]:
response = chain.invoke({"question": "How many employees are there"})
response

'SELECT COUNT(*) AS total_employees FROM customer;'

In [11]:
db.run(response)

'[(1000,)]'

In [12]:
response = chain.invoke({"question": "How many customers are from Bronx"})
print(response)
print(db.run(response))

SELECT COUNT(*) 
FROM customer 
WHERE city = 'Bronx'
LIMIT 5;
[(2,)]


In [13]:
response = chain.invoke({"question": "Which state are customers most from?"})
print(response)
print(db.run(response))

SELECT "state_", COUNT(*) as state_count
FROM customer
GROUP BY "state_"
ORDER BY state_count DESC
LIMIT 1;
[('CA', 79)]


In [15]:
response = chain.invoke({"question": "Give me the top 5 customers who paid the highest shipping charges"})
print(response)
print(db.run(response))

SELECT c.customerid, c.firstname, c.lastname, s.shipping
FROM customer c
JOIN sale s ON c.customerid = s.customerid
ORDER BY s.shipping DESC
LIMIT 5;
[(1096, 'Meliha', 'Handzic', 59.37), (1950, 'Jack', 'Freund', 54.45), (724, 'Epaminondas', 'Kapetanios', 54.41), (3979, 'Glen', 'Quintana', 54.39), (1385, 'Peter', 'Pille', 52.88)]
